In [1]:
# Make HTTP requests
import requests
# Scrape data from an HTML document
from bs4 import BeautifulSoup
# I/O
import os
# Search and manipulate strings
import re

In [2]:
client_id = "yxeLnXDdwoDJunEPDRUgYVmH-I703sGZKWGoCdKbFNupYKHugalkOVqtzh3aSe82"
client_secret = "zd3utRAeYAprwImrgEPERa6BPjNrJMOcg4ogsbTAR-uCoIoh9zu28ufhMVRYlBTpb8kSHkWK75LDrHSerdEOag"
client_access_token = "wVsdPF-_vbLLypQZ3qaR0EwytwJW2jKBsK-PoZwv6BDKQLDK0gkUwstOYuH-EDD2"

In [27]:
# Get artist object from Genius API
def request_artist_info(artist_name, page):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + client_access_token}
    search_url = base_url + '/search?per_page=10&page=' + str(page)
    data = {'q': artist_name}
    response = requests.get(search_url, data = data, headers = headers)
    return response

#request_artist_info("Chappell Roan", 1)
def request_song_url(artist_name, num_songs):
    page = 1
    urls = []
    
    while True:
        response = request_artist_info(artist_name, page)
        json = response.json()
        
        # Collect up to num_songs song objects from artist
        urls_info = []
        for hit in json['response']['hits']:
            if artist_name.lower() in hit['result']['primary_artist']['name'].lower():
                urls_info.append(hit)
    
        # Collect song URL's from song objects
        for info in urls_info:
            if (len(urls) < num_songs):
                url = info['result']['url']
                urls.append(url)
            
        if (len(urls) == num_songs):
            break
        else:
            page += 1

    # Now find each song title from each url
    track_titles = []
    for url in urls:
        match = re.search(r'The-beatles-(.*?)-lyrics', url)
        
        if match:
            # Save each title
            title = match.group(1)
    
            # Append to track_titles
            track_titles.append(title) 
           
    return [urls, track_titles]

[urls, track_titles] = request_song_url("The Beatles", 200)

In [28]:
# Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    song = requests.get(url)
    html = BeautifulSoup(song.content, 'html.parser')
    lyrics_divs = html.find_all("div", {"data-lyrics-container": "true"})
    
    if lyrics_divs:
        lyrics = "\n".join(div.get_text(separator="\n") for div in lyrics_divs)
    else:
        print("Lyrics not found!")
        
    # Remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics, flags=re.DOTALL).strip()
    
    # Remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])  
    
    return lyrics

In [30]:
def write_lyrics_to_corpus(urls, track_names, path):
    for i in range(len(urls)):
        # Get url and title
        url = urls[i]
        title = track_titles[i]
    
        # Create file, mypath, and filename
        file = str(title) + ".txt"
        filename = mypath + "/" + file
    
        # Open file to write to
        my_file = open(filename, "w", encoding = "utf-8")
    
        # Find lyrics
        lyrics = scrape_song_lyrics(url)
        my_file.write(lyrics)
        my_file.close()

mypath = "C:/Users/maede/Downloads/Python/INFO5653/Project/Project_Part1/Lyrics_Corpus"
write_lyrics_to_corpus(urls, track_titles, mypath)

Lyrics not found!


UnboundLocalError: local variable 'lyrics' referenced before assignment